# Stock Exchange Crawling and Analysis

## Project Description

This project is a key component of my research, leveraging the comprehensive financial dataset provided by StockAnalysis (https://stockanalysis.com/). Their rich collection of data from various stock exchanges worldwide has been instrumental in supporting this research. The project's goal is to develop a system capable of extracting data from income statements, balance sheets, and financial ratios for companies listed on stock exchanges globally

## Instruction


Import important library

In [ ]:
pip install vnstock3
import pandas as pd
import warnings
import requests
from bs4 import BeautifulSoup
from lxml import etree
import os
import datetime
import yfinance as yf
import json
from vnstock3 import Vnstock
from concurrent.futures import ThreadPoolExecutor, as_completed
warnings.simplefilter(action='ignore', category=FutureWarning)

### Exchange Extraction

At first, we need to crawling the list of exchanges, which will help us on crawling list of companies inside.

In [2]:
data_source = "https://stockanalysis.com/list/exchanges/"
html = requests.get(data_source)
list_ex = pd.read_html(html.text)
df = list_ex[0]
df

,Exchange Name,Country,Code,Currency,Stocks
0,US OTC,United States,OTC,USD,7591
1,Bombay Stock Exchange,India,BOM,INR,4253
2,Tokyo Stock Exchange,Japan,TYO,JPY,3892
3,Nasdaq Stock Market,United States,NASDAQ,USD,3316
4,London Stock Exchange,United Kingdom,LON,GBP,3305
...,...,...,...,...,...
110,Beirut Stock Exchange,Lebanon,BDB,LBP,8
111,CBOE Europe,Europe,CHIX,EUR,6
112,Bratislava Stock Exchange,Slovakia,BSSE,EUR,6
113,Zimbabwe Stock Exchange,Zimbabwe,ZMSE,ZWL,5


After getting the list of exchanges as a DataFrame, I extract the access URL of these.

In [3]:
tree = etree.HTML(html.text)
href_list = tree.xpath('//*[@id="main"]/div/div/table/tbody/tr/td/a/@href')
href_list

['/list/otc-stocks/',
 '/list/bse-india/',
 '/list/tokyo-stock-exchange/',
 '/list/nasdaq-stocks/',
 '/list/london-stock-exchange/',
 '/list/shenzhen-stock-exchange/',
 '/list/hong-kong-stock-exchange/',
 '/list/nse-india/',
 '/list/shanghai-stock-exchange/',
 '/list/nyse-stocks/',
 '/list/australian-securities-exchange/',
 '/list/kosdaq-korea/',
 '/list/tsx-venture-exchange/',
 '/list/taipei-exchange/',
 '/list/taiwan-stock-exchange/',
 '/list/bursa-malaysia/',
 '/list/indonesia-stock-exchange/',
 '/list/stock-exchange-of-thailand/',
 '/list/korea-stock-exchange/',
 '/list/nasdaq-stockholm/',
 '/list/euronext-paris/',
 '/list/warsaw-stock-exchange/',
 '/list/toronto-stock-exchange/',
 '/list/deutsche-boerse-xetra/',
 '/list/london-stock-exchange-aim/',
 '/list/canadian-securities-exchange/',
 '/list/borsa-istanbul/',
 '/list/tel-aviv-stock-exchange/',
 '/list/pakistan-stock-exchange/',
 '/list/ho-chi-minh-stock-exchange/',
 '/list/singapore-exchange/',
 '/list/dhaka-stock-exchange/',


In [4]:
df["Url"] = href_list

In [5]:
df.to_csv("exchange_list.csv")

In this steps, we defines a list of exchanges we want, based on their Code. In example, I want to get HongKong Exchange, I do put it in the list.

In [6]:
ex_to_get = ["HKG"]

In [7]:
for ex in ex_to_get:
    print(f"https://stockanalysis.com{df[df["Code"] == ex]["Url"].values[0]}")
    comp_html = requests.get(f"https://stockanalysis.com{df[df["Code"] == ex]["Url"].values[0]}")
    comp_list = pd.read_html(comp_html.text)
    tree = etree.HTML(comp_html.text)
    href_list = tree.xpath('//*[@id="main-table"]/tbody/tr/td[2]/a/@href')
    comp_list[0]["Url"] = href_list

    output_dir = f"./exchange/{ex}/"
    os.makedirs(output_dir, exist_ok=True)

    comp_list[0].to_csv(f"./exchange/{ex}/{ex}.csv")


https://stockanalysis.com/list/hong-kong-stock-exchange/


### Companies List

This code does will extract their full of companies inside, containing Order, Symbol, Display Name, Market Capitalization,... of the company. It represent this list as a DataFrame (https://stockanalysis.com/list/hong-kong-stock-exchange/). The file will be saved in 'exchange' directory, eg. exchange/HKG/HKG.csv, the .csv file is list of companies which is registered in HongKong Exchange.

In [8]:
for ex in ex_to_get:
    comp_list = requests.get(f"https://stockanalysis.com/api/screener/a/f?m=marketCap&s=desc&c=no,s,n,marketCap,price,change,revenue&sc=marketCap&cn=3000&f=exchangeCode-is-{ex},subtype-is-stock&p=1&i=symbols")
    stock_json = json.loads(comp_list.text)
    stock = stock_json["data"]["data"]
    stock_df = pd.DataFrame(stock)
    print(stock_df)
    stock_df["url"] = "quote/"+stock_df["s"]
    print(stock_df)

    output_dir = f"./exchange/{ex}/"
    os.makedirs(output_dir, exist_ok=True)

    stock_df.to_csv(f"./exchange/{ex}/{ex}.csv")
    


        no         s                                                n  \
0        1  hkg/0700                         Tencent Holdings Limited   
1        2  hkg/9988                    Alibaba Group Holding Limited   
2        3  hkg/1398  Industrial and Commercial Bank of China Limited   
3        4  hkg/1288               Agricultural Bank of China Limited   
4        5  hkg/4333                              Cisco Systems, Inc.   
...    ...       ...                                              ...   
2549  2550  hkg/8422                        WT Group Holdings Limited   
2550  2551  hkg/8191         Hong Wei (Asia) Holdings Company Limited   
2551  2552  hkg/8431           Hao Bai International (Cayman) Limited   
2552  2553  hkg/8437                             RMH Holdings Limited   
2553  2554  hkg/8456           Mansion International Holdings Limited   

          marketCap    price  change       revenue  
0     4710774058571  521.500    2.76  7.122120e+11  
1     24784644745

### Financial Indicators Extraction

When we observe the a Financial Tab in a company(Eg. https://stockanalysis.com/quote/hkg/0002/financials/?p=quarterly), we can see three tabs Income, Balance Sheet, Cash Flow and Ratios. In my scope, I crawled all the tabs except Cash Flow. Each company has its own directory, the directory's name is the Symbol of a company, and inside is .csv file for those documents.

In [ ]:
def fetch_company_data(row):
    try:
        url = "https://stockanalysis.com/"
        comp_url = row["url"]  # Get the URL from the DataFrame
        symbol = row["s"].split('/')[1]  # Get the symbol
        print(f"Fetching company: {symbol}")
        output_dir = f"./exchange/{ex}/{symbol}"
        os.makedirs(output_dir, exist_ok=True)

        # Fetch financial data
        income_stm_html = requests.get(f"{url}{comp_url}/financials/?p=quarterly")
        balance_sheet_html = requests.get(f"{url}{comp_url}/financials/balance-sheet/?p=quarterly")
        ratio_html = requests.get(f"{url}{comp_url}/financials/ratios/?p=quarterly")

        # Parse HTML tables into DataFrames
        income_stm = pd.read_html(income_stm_html.text)[0]
        balance_sheet = pd.read_html(balance_sheet_html.text)[0]
        ratio = pd.read_html(ratio_html.text)[0]
        
        # Save data to CSV files
        income_stm.to_csv(f"{output_dir}/income.csv")
        print(f"Income: Worked for {symbol}")
        balance_sheet.to_csv(f"{output_dir}/balance.csv")
        print(f"Balance: Worked for {symbol}")
        ratio.to_csv(f"{output_dir}/ratio.csv")
        print(f"Ratio: Worked for {symbol}")
    except Exception as e:
        print(f"Fetch company information error for {symbol}: {e}")

# Main script
ex = "HKG"
comp_df = pd.read_csv(f'./exchange/{ex}/{ex}.csv')
comp_list = comp_df[["s", "n", "url"]]

# Use ThreadPoolExecutor to multithread the fetching process
with ThreadPoolExecutor(max_workers=10) as executor:  # Adjust max_workers as needed
    futures = [executor.submit(fetch_company_data, row) for _, row in comp_list.iterrows()]

    # Wait for all futures to complete
    for future in as_completed(futures):
        future.result()  # Handle exceptions if needed

Fetching company: 0700
Fetching company: 9988
Fetching company: 1398
Fetching company: 1288
Fetching company: 4333
Fetching company: 0941
Fetching company: 0939
Fetching company: 3988
Fetching company: 0005
Fetching company: 0857
Income: Worked for 0939
Balance: Worked for 0939
Ratio: Worked for 0939
Fetching company: 1810
Income: Worked for 4333
Balance: Worked for 4333
Ratio: Worked for 4333
Fetching company: 3968
Income: Worked for 1398
Balance: Worked for 1398
Ratio: Worked for 1398
Fetching company: 1211
Income: Worked for 9988
Balance: Worked for 9988
Ratio: Worked for 9988
Fetching company: 3690
Income: Worked for 0005
Balance: Worked for 0005
Ratio: Worked for 0005
Fetching company: 2628
Income: Worked for 0700
Balance: Worked for 0700
Ratio: Worked for 0700
Fetching company: 4336
Income: Worked for 1288
Balance: Worked for 1288
Ratio: Worked for 1288
Fetching company: 2318
Income: Worked for 0857
Balance: Worked for 0857
Ratio: Worked for 0857
Income: Worked for 0941
Fetching 

### Stock price Extraction

After getting the financial indicators from those companies, we need to extract the price at that time, I used Yfinance library to extract the price for companies except Vietnam's companies (Vietnam's companies method is demonstrated below)

In [ ]:
ex = "HKG"  # Shenzhen Stock Exchange
comp_df = pd.read_csv(f'./exchange/{ex}/{ex}.csv')
comp_list = comp_df[["s", "n", "url"]]

# List of quarter-end dates
quarters = [
    "2024-09-30", "2024-06-30", "2024-03-31", "2023-12-31", "2023-09-30",
    "2023-06-30", "2023-03-31", "2022-12-31", "2022-09-30", "2022-06-30",
    "2022-03-31", "2021-12-31", "2021-09-30", "2021-06-30", "2021-03-31",
    "2020-12-31", "2020-09-30", "2020-06-30", "2020-03-31", "2019-12-31"
]

for _, row in comp_list.iterrows():
    try:
        symbol = row["s"].split('/')[1] 
        stock_code = f"{str(symbol).zfill(6)}.SZ"  # Ensure Shenzhen stocks have .SZ suffix
        print(f"Fetching data for: {stock_code}")

        # Fetch historical stock prices
        try:
            stock_data = yf.download(stock_code, start="2019-12-01", end="2024-10-01", interval="1d", rounding=True)["Close"]
        except:
            print("Error in yf download")
        # Skip if no data
        if stock_data.empty:
            print(f"No data found for {stock_code}, skipping...")
            continue

        # Convert index to string format
        # Extract quarter-end prices
        data = []
        for date in quarters:
            date = pd.to_datetime(date)
            if date in stock_data.index:
                price = stock_data.loc[date][0]  # This is already a scalar value if stock_data is a Series
            else:
                price = stock_data.asof(date)  # Returns a Series, need to extract the value
                if isinstance(price, pd.Series):  
                    price = price.iloc[0]  # Extract the first value if it's a Series
            print(price)
            data.append([symbol, date, price])


        # print(data)
        # Convert to DataFrame
        df = pd.DataFrame(data, columns=["Stock Symbol", "Date", "Stock Price"])

        # Ensure directory exists
        folder_path = f'./exchange/{ex}/{symbol}/'
        os.makedirs(folder_path, exist_ok=True)

        # Save to CSV
        df.to_csv(f'{folder_path}price.csv', index=False)
        print(f"Saved price data for {symbol}")

    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")


62.45
51.3
60.75
62.65
65.98
63.72
69.32
68.53
56.71
64.95
55.96
67.35
61.85
72.91
55.75
60.83
37.68
24.63
16.71
19.9
Saved price data for 300124
Fetching data for: 300999.SZ
35.49
27.35
30.07
33.27
34.26
39.85
42.87
43.33
42.96
53.64
49.23
62.49
68.93
84.36
76.83
107.41
nan
nan
nan
nan


[*********************100%***********************]  1 of 1 completed


Saved price data for 300999
Fetching data for: 003816.SZ
4.5
4.5436
3.9647
3.052
3.0814
2.9666
2.7377
2.566
2.5183
2.5925
2.5277
2.898
3.1202
2.398
2.5507
2.5328
2.5687
2.5971
2.5444
3.1586
Saved price data for 003816
Fetching data for: 000063.SZ


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


31.15
27.3
27.32
25.85
31.9
44.45
31.38
24.92
20.62
24.6
22.73
31.87
31.51
31.43
27.74
31.83
31.3
37.76
40.27
33.3
Saved price data for 000063
Fetching data for: 002142.SZ
25.7
21.45
20.06
19.55
26.13
24.1
26.02


...

1.52
0.8
1.67
2.9
2.87
3.1
3.57
3.52
3.46
4.15
4.13
3.7
4.19
4.8175
3.9198
3.9697
4.5582
2.7503
3.3183
3.7269
Saved price data for 300117
Fetching data for: 200512.SZ


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


3.66
3.0
2.8336
2.8795
2.9162
3.0445
3.0244
2.9995
2.8416
2.9912
3.0975
3.4918
3.6768
3.532
3.9218
4.0914
2.82
2.2884
2.2905
2.8299
Saved price data for 200512
Fetching data for: 002750.SZ
2.27
1.48
7.48
10.0
10.64
9.27
10.67
11.13
9.93
11.93
16.9
16.13
7.19
7.51
8.1
8.38
7.46
9.22
9.46
12.87
Saved price data for 002750
Fetching data for: 200992.SZ


[*********************100%***********************]  1 of 1 completed

1.9
1.57
1.99
2.12
2.15
2.42
2.68
2.83
2.73
2.96
2.89
3.04
3.09
2.96
3.19
3.2
3.39
2.77
2.64
3.06
Saved price data for 200992


### Vietnam's Company Stock Price Extraction

In this example, I tested with Hanoi and Ho Chi Minh Stock Exchange, using vnstock3 library to do the extraction.

In [ ]:
ex = "HNX"  # Shenzhen Stock Exchange
comp_df = pd.read_csv(f'./exchange/{ex}/{ex}.csv')
comp_list = comp_df[["s", "n", "url"]]

# List of quarter-end dates
quarters = [
    "2024-09-30", "2024-06-30", "2024-03-31", "2023-12-31", "2023-09-30",
    "2023-06-30", "2023-03-31", "2022-12-31", "2022-09-30", "2022-06-30",
    "2022-03-31", "2021-12-31", "2021-09-30", "2021-06-30", "2021-03-31",
    "2020-12-31", "2020-09-30", "2020-06-30", "2020-03-31", "2019-12-31"
]

for _, row in comp_list.iterrows():
    try:
        symbol = row["s"].split('/')[1]
        print(f"Fetching data for: {symbol}")
        stock = Vnstock().stock(symbol=symbol, source='VCI')
        # Fetch historical stock prices
        try:
            stock_data = stock.quote.history(start='2019-12-31', end='2024-10-01')[["time","close"]]
        except:
            print("Error in yf download")
        stock_data.reset_index
        stock_data.set_index("time", inplace=True)
        print(stock_data.columns)
        
        # Skip if no data
        if stock_data.empty:
            print(f"No data found for {symbol}, skipping...")
            continue

        # Convert index to string format
        # Extract quarter-end prices
        data = []
        for date in quarters:
            if date in stock_data.index:
                print("Stock has date")
                price = stock_data.loc[date]["close"]
                data.append([symbol, date, price])
            else:
                nearest_date = stock_data.index.asof(date)
                nearest_price = stock_data.loc[nearest_date]["close"]
                print(f"Stock get nearest date: from {date} to {nearest_date} with price {nearest_price}")
                data.append([symbol, date, nearest_price])

        print(data)
        # Convert to DataFrame
        df = pd.DataFrame(data, columns=["Stock Symbol", "Date", "Stock Price"])

        # Ensure directory exists
        folder_path = f'./exchange/{ex}/{symbol}/'
        os.makedirs(folder_path, exist_ok=True)

        # Save to CSV
        df.to_csv(f'{folder_path}price.csv', index=False)
        print(f"Saved price data for {symbol}")

    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
